In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split

In [24]:
m_cols = ['movieId', 'title', 'release_date', 'video_release_date', 'imdb_url']
movies = pd.read_csv('/content/drive/MyDrive/ml-100k/u.item', sep='|', names=m_cols, usecols=range(5),encoding='latin-1')

r_cols = ['userId', 'movieId', 'rating', 'unix_timestamp']
Ratings = pd.read_csv('/content/drive/MyDrive/ml-100k/u.data', sep='\t', names=r_cols,encoding='latin-1')

Ratings.head()

,userId,movieId,rating,unix_timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [3]:
count = Ratings.groupby(by="movieId",as_index=False)['rating'].count()
Ratings = pd.merge(Ratings,count,on='movieId')
Ratings.columns =['userId', 'movieId', 'rating', 'unix_timestamp','No.ratings'] 
print(Ratings.shape) 
Ratings.head()

(100000, 5)


,userId,movieId,rating,unix_timestamp,No.ratings
0,196,242,3,881250949,117
1,63,242,3,875747190,117
2,226,242,5,883888671,117
3,154,242,3,879138235,117
4,306,242,5,876503793,117


In [5]:
Ratings.drop(Ratings.loc[Ratings['No.ratings']<3].index, inplace=True) 
print(Ratings.shape) 
Ratings.head()

(99723, 5)


,userId,movieId,rating,unix_timestamp,No.ratings
0,196,242,3,881250949,117
1,63,242,3,875747190,117
2,226,242,5,883888671,117
3,154,242,3,879138235,117
4,306,242,5,876503793,117


In [10]:
train, test = train_test_split(Ratings, test_size=0.05)
# print("\ntrain\n",train.head())
print("Train num.",train.shape)
print("Test num.",test.shape)

Train num. (94736, 5)
Test num. (4987, 5)


In [13]:
Mean = train.groupby(by="userId",as_index=False)['rating'].mean() 
Rating_avg = pd.merge(train,Mean,on='userId')
Rating_avg['adg_rating']=Rating_avg['rating_x']-Rating_avg['rating_y'] 
Rating_avg.head()

,userId,movieId,rating_x,unix_timestamp,No.ratings,rating_y,adg_rating
0,804,31,4,879442792,154,3.642405,0.357595
1,804,708,3,879445783,101,3.642405,-0.642405
2,804,380,4,879445715,116,3.642405,0.357595
3,804,24,5,879443776,174,3.642405,1.357595
4,804,1101,3,879444805,74,3.642405,-0.642405


In [14]:
umRate = pd.pivot_table(Rating_avg,values='rating_x',index='userId',columns='movieId')
final = pd.pivot_table(Rating_avg,values='adg_rating',index='userId',columns='movieId') 
print("\nUser-Movie dataset:\n")
final.head()


User-Movie dataset:



movieId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,1528,1529,1530,1531,1534,1535,1537,1538,1539,1540,1544,1545,1552,1553,1555,1558,1560,1589,1591,1592,1597,1598,1600,1602,1605,1607,1608,1609,1610,1612,1615,1620,1622,1623,1628,1639,1643,1652,1658,1664
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1.423664,-0.576336,0.423664,-0.576336,-0.576336,1.423664,0.423664,-2.576336,1.423664,-0.576336,-1.576336,1.423664,1.423664,1.423664,1.423664,1.423664,-0.576336,0.423664,1.423664,0.423664,-2.576336,0.423664,0.423664,-0.576336,0.423664,-0.576336,-1.576336,0.423664,-2.576336,-0.576336,-0.576336,1.423664,0.423664,-1.576336,-2.576336,-1.576336,-1.576336,-0.576336,0.423664,-0.576336,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.338983,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.661017,NaN,NaN,0.338983,0.338983,NaN,NaN,NaN,NaN,-0.661017,NaN,NaN,NaN,NaN,NaN,0.338983,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.333333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1.157576,0.157576,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.157576,NaN,NaN,NaN,0.157576,NaN,NaN,1.157576,0.157576,NaN,NaN,NaN,1.157576,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.157576,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
final_movie = final.fillna(final.mean(axis=0)) #fill nan values by movie Average
print("\nfinal movie\n",final_movie.head())
final_user = final.apply(lambda row: row.fillna(row.mean()), axis=1) #fill nan values by user Average
print("\nfinal User \n",final_user.head())


final movie
 movieId      1         2         3     ...      1652      1658      1664
userId                                 ...                              
1        1.423664 -0.576336  0.423664  ... -1.617784 -0.144713  0.037173
2        0.338983 -0.267573 -0.441795  ... -1.617784 -0.144713  0.037173
3        0.297425 -0.267573 -0.441795  ... -1.617784 -0.144713  0.037173
4        0.297425 -0.267573 -0.441795  ... -1.617784 -0.144713  0.037173
5        1.157576  0.157576 -0.441795  ... -1.617784 -0.144713  0.037173

[5 rows x 1473 columns]

final User 
 movieId          1             2     ...          1658          1664
userId                               ...                            
1        1.423664e+00 -5.763359e-01  ... -1.966197e-16 -1.966197e-16
2        3.389831e-01 -6.021549e-17  ... -6.021549e-17 -6.021549e-17
3        2.178551e-16  2.178551e-16  ...  2.178551e-16  2.178551e-16
4        2.960595e-16  2.960595e-16  ...  2.960595e-16  2.960595e-16
5        1.157576e+00 

In [29]:
# user similarity on replacing NAN by user avg
b = cosine_similarity(final_user) 
np.fill_diagonal(b, 0 ) 
similarity_with_user = pd.DataFrame(b,index=final_user.index)
similarity_with_user.columns=final_user.index  # similarity user-user table
print("\nsimilarity_with_user\n", similarity_with_user.head())


similarity_with_user
 userId       1         2             3    ...       941       942       943
userId                                    ...                              
1       0.000000  0.038798  1.052674e-02  ...  0.029661 -0.036753  0.026367
2       0.038798  0.000000  1.184174e-02  ... -0.016635  0.012019  0.026389
3       0.010527  0.011842  0.000000e+00  ...  0.034463 -0.010378  0.001846
4       0.059315 -0.012316 -6.240173e-02  ...  0.258650  0.072483 -0.020162
5       0.139597  0.041727 -6.011898e-31  ... -0.020130  0.048891  0.077035

[5 rows x 943 columns]


In [30]:
# user similarity on replacing NAN by item(movie) avg
cosine = cosine_similarity(final_movie)
np.fill_diagonal(cosine, 0 )
similarity_with_movie = pd.DataFrame(cosine,index=final_movie.index)
similarity_with_movie.columns=final_user.index # similarity user-user table
print("\nsimilarity_with_movie\n", similarity_with_movie.head()) 


similarity_with_movie
 userId       1         2         3    ...       941       942       943
userId                                ...                              
1       0.000000  0.803935  0.781588  ...  0.825049  0.789454  0.684266
2       0.803935  0.000000  0.902098  ...  0.948287  0.928520  0.813870
3       0.781588  0.902098  0.000000  ...  0.926803  0.897666  0.790888
4       0.822330  0.941755  0.917930  ...  0.975484  0.951192  0.828007
5       0.712219  0.802578  0.777933  ...  0.818488  0.801475  0.708214

[5 rows x 943 columns]


In [31]:
def find_n_neighbours(df,k):
    df = df.apply(lambda x: pd.Series(x.sort_values(ascending=False)
           .iloc[:k].index, 
          index=['top{}'.format(i) for i in range(1, k+1)]), axis=1)
    return df

In [32]:
# top 20 neighbours for each user
sim_user_20_u = find_n_neighbours(similarity_with_user,20)
print("\n20 neighbours on user avg\n",sim_user_20_u.head())
sim_user_20_m = find_n_neighbours(similarity_with_movie,20)
print("\n20 neighbours on movie avg\n", sim_user_20_m.head())


20 neighbours on user avg
         top1  top2  top3  top4  top5  ...  top16  top17  top18  top19  top20
userId                                ...                                   
1        868   773   592   276    44  ...    429    487     13    661    344
2        651    34   743   296   310  ...    460    258     33    937    444
3        687   155   808   284   656  ...    445     46    710    547    369
4         35   841   940   353   431  ...    462    510    362    464    105
5        497   222   276   650   457  ...    776    653    660    643      8

[5 rows x 20 columns]

20 neighbours on movie avg
         top1  top2  top3  top4  top5  ...  top16  top17  top18  top19  top20
userId                                ...                                   
1        225   822   549   895   800  ...     27    376    920    247    691
2        384    33   849   800   728  ...    909    163    400    140    594
3         33   284   155   810    47  ...    140    866    728    888    

In [18]:
def get_user_similar_movies(user1, user2):
    common_movies = Rating_avg[Rating_avg.userId == user1].merge(
    Rating_avg[Rating_avg.userId == user2],
    on = "movieId",
    how = "inner" )
    return common_movies.merge( movies, on = 'movieId' )


def User_item_score(user,item): 
    a = sim_user_20_m [sim_user_20_m.index==user].values
    b = a.squeeze().tolist()
    c = final_movie.loc[:,item] #all user ratings of movie Item
    d = c[c.index.isin(b)]      #keep neighbours rating from c
    f = d[d.notnull()]          #keep not null ratings
    avg_user = Mean.loc[Mean['userId'] == user,'rating'].values[0]
    index = f.index.values.squeeze().tolist()
    corr = similarity_with_movie.loc[user,index]
    fin = pd.concat([f, corr], axis=1)
    fin.columns = ['adg_score','correlation']
    fin['score']=fin.apply(lambda x:x['adg_score'] * x['correlation'],axis=1)
    nume = fin['score'].sum()
    deno = fin['correlation'].sum()
    final_score = avg_user + (nume/deno)
    return final_score

In [19]:
Rating_avg = Rating_avg.astype({"movieId": str})
Movie_user = Rating_avg.groupby(by = 'userId')['movieId'].apply(lambda x:','.join(x)) # the movies user seen

In [20]:
def top_n_rec(user, n): #function to predict user for unseen movie
    Movie_seen_by_user = umRate.columns[umRate[umRate.index==user].notna().any()].tolist()
    a = sim_user_20_m[sim_user_20_m.index==user].values
    b = a.squeeze().tolist()
    d = Movie_user[Movie_user.index.isin(b)]
    l = ','.join(d.values)
    Movie_seen_by_similar_users = l.split(',')
    Movies_under_consideration = list(set(Movie_seen_by_similar_users)-set(list(map(str, Movie_seen_by_user))))
    Movies_under_consideration = list(map(int, Movies_under_consideration))
    score = []
    for item in Movies_under_consideration:
        score.append(User_item_score(user,item))
    data = pd.DataFrame({'movieId':Movies_under_consideration,'score':score})
    top_n_recommendation = data.sort_values(by='score',ascending=False).head((n))
    Movie_Name = top_n_recommendation.merge(movies, how='inner', on='movieId')
    Movie_Names = Movie_Name.title.values.tolist()
    return Movie_Names

In [34]:
user = int(input("\nuser id: ")) 
predicted_movies = top_n_rec(user,10) 
print("n Recommendations: \n") 
for i in predicted_movies:
    print(i)

# o = input("\npress enter to continue ")


user id: 1
n Recommendations: 

Close Shave, A (1995)
Schindler's List (1993)
Titanic (1997)
Casablanca (1942)
Boot, Das (1981)
L.A. Confidential (1997)
As Good As It Gets (1997)
Rear Window (1954)
Silence of the Lambs, The (1991)
Secrets & Lies (1996)


In [36]:
user_id=test.iloc[:,0].values
movies_id=test.iloc[:,1].values
rate=test.iloc[:,2].values
score_test=[]
shift=[]
sum=0

for i in range(0,len(test)):
    pred_score=User_item_score(user_id[i],movies_id[i])
    pred_shift=rate[i]-pred_score
    shift.append(pred_shift)
    sum=sum+pred_shift
    mean=sum/(i+1)
    score_test.append(pred_score)
test['rate_pred']=score_test
test['pred_shift']=shift
# print("test\n",test.head())
print("Mean Prediction shift = ", mean)

Mean Prediction shift =  0.027667325503723594
